<h1>MODELLING SWISS VOTATION OUTCOMES</h1>
<h3>Coursera Capstone Report for the IBM Professional Certificate in Data Science</h3><br>
Juliane Klatt
<hr>
<h2>Table of Contents</h2>
<ol>
    <li>Introduction</li>
    <li>Data</li>
        <ol>
            <li>Swiss Plebiscite Data</li>
            <li>List of Swiss Municipalities</li>
            <li>Foursquare Location Data</li>
        </ol>
    <li>Methodology</li>
    <li>Analysis</li>
    <li>Results</li>
    <li>Conclusion</li>
    <li>References</li>
</ol>

<hr>
<h2>1. Introduction</h2>

<p>In Switzerland Swiss voters can demand a binding referendum at federal, cantonal, and municipal level. They are a central feature of Swiss political life. It is not the government's choice whether or when a referendum is held, but it is a legal procedure regulated by the Swiss constitution. There are two types of referendums: optional and mandatory referendums. Any federal law, certain other federal resolutions, and international treaties that are ongoing in nature, or any change to Swiss law may be subject to an optional referendum. Such referendum takes place if at least 50,000 people or eight cantons have petitioned to do so within 100 days. Mandatory referendums take place on any amendment to the constitution and on any joining of a multinational community or organization for collective security. Constitutional amendments are proposed by the parliament or by the cantons or by federal popular initiative.</p>
<p>In this project we will investigate whether canton-level outcomes of Swiss referendums are correlated with the type and quantity of venues present in a canton. Once correlations have been identified, several classification models will be built and compared in their ability to correctly predict swiss votation outcomes by canton based on the canton's venues.</p>

<hr>
<h2>2. Data</h2>
<ul>
<li><a href="https://www.bk.admin.ch/bk/de/home/politische-rechte/volksabstimmungen.html">Swiss plebiscite data</a>
<li><a href="https://en.wikipedia.org/wiki/List_of_municipalities_of_Switzerland">List of Swiss municipalities</a>
<li><a href="https://foursquare.com/">Foursquare location data</a>
</ul>
<h3>2.A. Swiss Plebiscite Data</h3>
<a href="https://www.bk.admin.ch/bk/de/home/politische-rechte/volksabstimmungen.html">Source</a>
<h3>2.B. Swiss Municipalities by Canton and Georeferences</h3>
<a href="https://en.wikipedia.org/wiki/List_of_municipalities_of_Switzerland">Source</a>
<h3>2.C. Foursquare Location Data</h3>
<a href="https://foursquare.com/">Source</a>

<hr>
<h2>3. Methodology</h2>

<hr>
<h2>4. Analysis</h2>

<hr>
<h2>5. Results</h2>

<hr>
<h2>6. Conclusion</h2>

<hr>
<h2>7. References</h2>